In [96]:
fator_req = 15

In [97]:
import warnings
warnings.filterwarnings('ignore')

In [98]:
import pandas as pd
from code_class import GeoVictoria, FolhaPonto
from tqdm import tqdm
import datetime as dt
import codeshot

pd.set_option('display.max_rows', 100)

In [99]:
dias_semana = [(   'Sunday',       'Domingo'),
               (   'Monday', 'Segunda-feira'),
               (  'Tuesday',   'Terça-feira'),
               ('Wednesday',  'Quarta-feira'),
               ( 'Thursday',  'Quinta-feira'),
               (   'Friday',   'Sexta-feira'),
               ( 'Saturday',        'Sábado')]

df_data = pd.DataFrame(data=[dt.date(year=2022, month=11, day=21) + dt.timedelta(days=x) for x in range(0, 182)], columns=['Data'])

In [100]:
lista_tp_permis = []

for permission in GeoVictoria.GetPermissionList():
    lista_tp_permis.append((permission['ID_TIPO_PERMISO'], permission['DESCRIPCION_TIPO_PERMISO']))

In [101]:
df_departamento = pd.read_excel('Z:\\Int logística\\Matheus\\BI DP\\Relaçao de funcionários.xlsx')

df_departamento['Funcionário'] = df_departamento['Funcionário'].apply(lambda x: '{:06d}'.format(x))

In [102]:
df_funcionários = pd.DataFrame(GeoVictoria.GetUserList())

df_funcionários['Setor'] = df_funcionários['GroupDescription'].apply(lambda x:  '' if 'LOGIN' in x.upper() else
                                                                                'Garantia' if 'GARAN' in x.upper() else
                                                                                'Logistica' if 'LOGIS' in x.upper() else
                                                                                'Logistica' if 'LOGÍS' in x.upper() else
                                                                                'Vendas' if 'VENDA' in x.upper() else
                                                                                'Compras' if 'COMPR' in x.upper() else
                                                                                'Gente e Gestão' if 'GENT' in x.upper() else
                                                                                'Gente e Gestão' if 'DP' in x.upper() else
                                                                                'Estoque' if 'ESTOQ' in x.upper() else
                                                                                'Expedição' if 'EXPED' in x.upper() else
                                                                                'Almoxarifado' if 'ALMOX' in x.upper() else
                                                                                'Financeiro' if 'FINAN' in x.upper() else
                                                                                'Servicos Gerais' if 'GERAIS' in x.upper() else
                                                                                'E-commerce' if 'COMMERCE' in x.upper() else
                                                                                'Inteligencia Comercial' if 'COMERCIAL' in x.upper() else
                                                                                'Cobranca' if 'COBRAN' in x.upper() else
                                                                                'Marketing' if 'MARKET' in x.upper() else
                                                                                'Dados' if 'DADOS' in x.upper() else
                                                                                'Administrativo' if 'ADM' in x.upper() else
                                                                                'Supervisor' if 'CHEFE' in x.upper() else
                                                                                'Tecnologia' if 'TECNO' in x.upper() else
                                                                                'Projetos' if 'PROJE' in x.upper() else
                                                                                'Expansão' if 'EXPAN' in x.upper() else
                                                                                'Motorista' if 'MOTOR' in x.upper() else '')

df_funcionários['Nome completo'] = df_funcionários['Name'] + ' ' + df_funcionários['LastName']

for i in df_funcionários.index:   
    nome = []
    for x in df_funcionários.loc[i, 'Nome completo'].split(' '):
        try:
            nome.append(x[0].upper() + x[1:])
        except:
            print(x)
        nome_completo = ' '.join(nome)
        df_funcionários.loc[i, 'Nome completo'] = nome_completo

cod_fun = list(df_funcionários['Identifier'])

In [103]:
df_funcionários = pd.merge(df_funcionários, df_departamento, how='outer', left_on=['Identifier'], right_on=['Funcionário'])

In [104]:
contador = 1

df_permis = pd.DataFrame(GeoVictoria.GetPermissions(person=','.join(cod_fun)))
df_horas = pd.DataFrame(GeoVictoria.GetAttendance(person=','.join(cod_fun[:fator_req])))

while contador < int(len(cod_fun) / fator_req) + 1:
    df_horas = df_horas.append(pd.DataFrame(GeoVictoria.GetAttendance(person=','.join(cod_fun[(contador * fator_req) + 1:(contador * fator_req) + fator_req]))))
    contador = contador + 1

df_horas = df_horas.reset_index(drop=True)

In [105]:
df_permis['Data inicial'] = df_permis['PERMISSION_START'].apply(lambda x: dt.date(int(x[:4]), int(x[4:6]), int(x[6:8])) if len(x) > 10 else None)
df_permis['Data final'] = df_permis['PERMISSION_END'].apply(lambda x: dt.date(int(x[:4]), int(x[4:6]), int(x[6:8])) if len(x) > 10 else None)

In [106]:
df_ponto = pd.DataFrame(columns=['Cod. Funcionario',
                                 'Nome completo',
                                 'Departamento',
                                 'Função',
                                 'Turno',
                                 'Permissao',
                                 'Data',
                                 'H. Ent1',
                                 'H. Sai1',
                                 'Intervalo',
                                 'H. Ent2',
                                 'H. Sai2'])

df_ponto['Cod. Funcionario'] = df_horas['identifier']
df_ponto['Data'] = df_horas['begins']
df_ponto['H. Ent1'] = df_horas['first_punch']
df_ponto['H. Sai1'] = df_horas['second_punch']
df_ponto['H. Ent2'] = df_horas['third_punch']
df_ponto['H. Sai2'] = df_horas['fourth_punch']

df_ponto['Data'] = df_ponto['Data'].apply(lambda x: dt.date(int(x[:4]), int(x[4:6]), int(x[6:8])) if len(x) > 10 else None)

df_ponto['H. Ent1'] = df_ponto['H. Ent1'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')
df_ponto['H. Sai1'] = df_ponto['H. Sai1'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')
df_ponto['H. Ent2'] = df_ponto['H. Ent2'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')
df_ponto['H. Sai2'] = df_ponto['H. Sai2'].apply(lambda x: f'{x[8:10]}:{x[10:12]}' if len(x) > 10 else '--:--')

df_ponto['Intervalo'] = df_ponto['H. Ent2'].apply(lambda x: ((int(x[:2]) * 60) + (int(x[3:5]))) if '--:--' != x else 0) - df_ponto['H. Sai1'].apply(lambda x: ((int(x[:2]) * 60) + (int(x[3:5]))) if '--:--' != x else 0)
df_ponto['Intervalo'] = df_ponto['Intervalo'].apply(lambda x: FolhaPonto.minuto_hora(x)).replace('00:00', '--:--')

for i in df_ponto.index:
    t_trabralhado = FolhaPonto.minuto_hora((FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Sai1']) - FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Ent1'])) + (FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Sai2']) - FolhaPonto.hora_minuto(df_ponto.loc[i, 'H. Ent2']))).replace('00:00', '--:--')

    if (df_ponto.loc[i, 'H. Ent1'] != '--:--' and df_ponto.loc[i, 'H. Sai1'] != '--:--') and df_ponto.loc[i, 'H. Ent2'] == '--:--':
        df_ponto.loc[i, 'Situacao'] = 'Esqueceu'
    elif (df_ponto.loc[i, 'H. Ent2'] != '--:--' and df_ponto.loc[i, 'H. Sai2'] != '--:--') and df_ponto.loc[i, 'H. Sai1'] == '--:--':
        df_ponto.loc[i, 'Situacao'] = 'Esqueceu'
    elif df_ponto.loc[i, 'H. Ent1'] != '--:--' and df_ponto.loc[i, 'H. Sai1'] != '--:--':
        df_ponto.loc[i, 'Situacao'] = 'Almoçou'
    elif df_ponto.loc[i, 'H. Ent1'] != '--:--' and df_ponto.loc[i, 'H. Sai2'] != '--:--':
        df_ponto.loc[i, 'Situacao'] = 'Saiu direto'
    else:
        df_ponto.loc[i, 'Situacao'] = '--:--'
        
    if df_ponto.loc[i, 'Situacao'] == 'Esqueceu':
        df_ponto.loc[i, 'Trabalhado'] = '--:--'
    elif df_ponto.loc[i, 'H. Sai2'] == '--:--':
        df_ponto.loc[i, 'Trabalhado'] = '--:--'
    else:
        df_ponto.loc[i, 'Trabalhado'] = t_trabralhado



    weekend_day = df_ponto.loc[i, 'Data'].strftime('%w')
    try:
        df_ponto.loc[i, 'Nome completo'] = df_funcionários[df_funcionários['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']]['Nome completo'].values[0]
        df_ponto.loc[i, 'Departamento'] = df_funcionários[df_funcionários['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']]['Departamento'].values[0]
        df_ponto.loc[i, 'Função'] = df_funcionários[df_funcionários['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']]['Função'].values[0]
        
        if (weekend_day != '0') & (weekend_day != '6'):
            df_ponto.loc[i, 'Turno'] = df_funcionários[df_funcionários['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']]['weeklyHoursCode'].values[0]
        else:
            df_ponto.loc[i, 'Turno'] = 'Nenhum'
            
        for (id, permis) in lista_tp_permis:
            if id == df_permis[(df_permis['Identifier'] == df_ponto.loc[i, 'Cod. Funcionario']) & (df_permis['Data inicial'] <= df_ponto.loc[i, 'Data']) & (df_permis['Data final'] >= df_ponto.loc[i, 'Data'])]['ID_PERMISSION_TYPE'].values[0]:
                if permis == None:
                    df_ponto.loc[i, 'Permissao'] = 'Nenhum'
                else:
                    df_ponto.loc[i, 'Permissao'] = permis

    except:
        df_ponto.loc[i, 'Permissao'] = 'Nenhum'
        continue

df_ponto = pd.merge(df_data, df_ponto, how='outer', on=['Data'])
df_ponto = df_ponto[df_ponto['Data'] <= dt.datetime.now().date()]
df_ponto = df_ponto.sort_values(by=['Data']).reset_index(drop=True)
df_ponto['Data'] = df_ponto['Data'].apply(lambda x: x.strftime('%d/%m/%Y'))
# df_ponto.iloc[:, 5:] = df_ponto.iloc[:, 5:].fillna('--:--')

In [107]:
for i in df_ponto.index:
    if df_ponto.loc[i, 'H. Sai2'] == '--:--':
        cumprimento = '--:--'
    elif (df_ponto.loc[i, 'Permissao'] == 'Nenhum') & (df_ponto.loc[i, 'Turno'] == '44 horas semanais'):
        cumprimento = '08:48'
    elif (df_ponto.loc[i, 'Permissao'] == 'Nenhum') & (df_ponto.loc[i, 'Turno'] == '35 horas semanais'):
        cumprimento = '07:00'
    elif (df_ponto.loc[i, 'Permissao'] == 'Nenhum') & (df_ponto.loc[i, 'Turno'] == '30 horas semanais '):
        cumprimento = '06:00'
    elif df_ponto.loc[i, 'Permissao'] == 'Atestado Parcial':
        cumprimento = FolhaPonto.minuto_hora(FolhaPonto.hora_minuto(df_ponto.loc[i, 'Trabalhado']))
    else:
        cumprimento = '--:--'

    df_ponto.loc[i, 'H. Extra / Faltante'] = FolhaPonto.minuto_hora(FolhaPonto.hora_minuto(df_ponto.loc[i, 'Trabalhado']) - FolhaPonto.hora_minuto(cumprimento)).replace('00:00','--:--')
    cumprimento = '--:--'

# df_ponto[(df_ponto['Cod. Funcionario'] == '002934')]# | (df_ponto['Cod. Funcionario'] == '004308')]

,Data,Cod. Funcionario,Nome completo,Departamento,Função,Turno,Permissao,H. Ent1,H. Sai1,Intervalo,H. Ent2,H. Sai2,Situacao,Trabalhado,H. Extra / Faltante
64,20/11/2022,002934,Carlos Eduardo Dos Santos Silva,Dados,Analista de Dados Jr,Nenhum,Nenhum,--:--,--:--,--:--,--:--,--:--,--:--,--:--,--:--
365,21/11/2022,002934,Carlos Eduardo Dos Santos Silva,Dados,Analista de Dados Jr,44 horas semanais,Nenhum,08:07,11:51,01:12,13:03,17:58,Almoçou,08:39,-00:09
664,22/11/2022,002934,Carlos Eduardo Dos Santos Silva,Dados,Analista de Dados Jr,44 horas semanais,Nenhum,07:55,12:57,01:12,14:09,18:10,Almoçou,09:03,00:15
966,23/11/2022,002934,Carlos Eduardo Dos Santos Silva,Dados,Analista de Dados Jr,44 horas semanais,Nenhum,07:48,12:12,01:14,13:26,18:14,Almoçou,09:12,00:24
1439,24/11/2022,002934,Carlos Eduardo Dos Santos Silva,Dados,Analista de Dados Jr,44 horas semanais,Nenhum,07:59,--:--,--:--,--:--,13:59,Saiu direto,06:00,-02:48
1585,25/11/2022,002934,Carlos Eduardo Dos Santos Silva,Dados,Analista de Dados Jr,44 horas semanais,Nenhum,07:56,11:43,01:08,12:51,18:09,Almoçou,09:05,00:17
1886,26/11/2022,002934,Carlos Eduardo Dos Santos Silva,Dados,Analista de Dados Jr,Nenhum,Nenhum,07:56,--:--,--:--,--:--,13:01,Saiu direto,05:05,05:05
2356,27/11/2022,002934,Carlos Eduardo Dos Santos Silva,Dados,Analista de Dados Jr,Nenhum,Nenhum,--:--,--:--,--:--,--:--,--:--,--:--,--:--,--:--
2489,28/11/2022,002934,Carlos Eduardo Dos Santos Silva,Dados,Analista de Dados Jr,44 horas semanais,Nenhum,07:56,12:14,02:40,14:54,18:04,Almoçou,07:28,-01:20
2956,29/11/2022,002934,Carlos Eduardo Dos Santos Silva,Dados,Analista de Dados Jr,44 horas semanais,Nenhum,07:55,12:15,01:03,13:18,18:07,Almoçou,09:09,00:21


In [108]:
df_ponto.loc[[62]][['H. Ent1', 'H. Sai1', 'H. Ent2', 'H. Sai2']].count()

H. Ent1    1
H. Sai1    1
H. Ent2    1
H. Sai2    1
dtype: int64

In [109]:
import json

json_diario = df_ponto.to_json(path_or_buf='ponto_diario.json',orient='records', indent=4)

In [110]:
FolhaPonto.calculo(df_ponto[(df_ponto['Cod. Funcionario'] == '002934')]['H. Extra / Faltante'])

'03:38'

In [111]:
GeoVictoria.GetPermissions(person='002944')

[{'ID_PERMISSION': 'maq8DgS1lRutankhwrrZAw',
  'ID_PERMISSION_TYPE': 'a3Afgu_jKdt56eNhPspj7g',
  'Identifier': '002944',
  'Email': 'ramon.mendes091@gmail.com',
  'PERMISSION_DESCRIPTION': ' ',
  'CREATED_BY': 'Taysa Chagas Parreira                                                                               ',
  'PERMISSION_END': '20221108235959',
  'PERMISSION_START': '20221108000000',
  'PERMISSION_START_HOUR': '',
  'PERMISSION_END_HOUR': '',
  'PERMISSION_HOURS': None,
  'ID_PERMISSION_EXTERNAL': None},
 {'ID_PERMISSION': 'rZAhObRRuNYNsHGlrTfVIw',
  'ID_PERMISSION_TYPE': '6iEjcqbzHc9ay1-yGoG5kQ',
  'Identifier': '002944',
  'Email': 'ramon.mendes091@gmail.com',
  'PERMISSION_DESCRIPTION': ' ',
  'CREATED_BY': 'Taysa Chagas Parreira                                                                               ',
  'PERMISSION_END': '20221221235959',
  'PERMISSION_START': '20221221000000',
  'PERMISSION_START_HOUR': '',
  'PERMISSION_END_HOUR': '',
  'PERMISSION_HOURS': None,
  'ID